In [45]:
import os
import pandas as pd
import re

In [46]:
data_folder = 'data/'
dataframes = []

In [47]:
def extract_currency_code(col):
    if col in ['Date', 'Year']:
        return col
    match = re.search(r'\((.*?)\)', col)
    if match:
        return match.group(1)
    return col

In [48]:
# Iterate over all CSV files in the data folder
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        
        # Extract the year from the filename (assuming the filename contains the year)
        year = filename.split('.')[0].split('_')[-1]  # Adjust this if the year is in a different part of the filename
        df['Year'] = int(year)
                # Convert the Date column based on the year
        if df['Year'].iloc[0] == 2023:
            df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
        else:
            df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')
        

        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file (optional)
# combined_df.to_csv('data/combined.csv', index=False)


In [49]:
combined_df.fillna(0, inplace=True)

In [50]:
combined_df.columns = [extract_currency_code(col) for col in combined_df.columns]

In [56]:
combined_df.loc[combined_df['Year'] == 2022]

,Date,DZD,AUD,BHD,VEF,BWP,BRL,BND,CAD,CLP,...,CHF,THB,TTD,TND,AED,GBP,USD,UYU,Year,VES
2483,3-Jan-22,138.6713,0.000000,0.0,0.0,0.000000,5.6303,1.3510,0.0000,844.69,...,0.91210,0.000,6.7742,0.0,3.6725,0.000000,1,44.614,2022,0.0
2484,4-Jan-22,138.9115,1.386770,0.0,0.0,0.000000,5.6770,1.3548,1.2708,851.43,...,0.91870,33.311,6.7723,0.0,3.6725,0.740988,1,44.725,2022,0.0
2485,5-Jan-22,138.8560,1.382743,0.0,0.0,11.792453,5.6622,1.3557,1.2717,852.03,...,0.91510,33.198,6.7495,0.0,3.6725,0.738498,1,44.731,2022,0.0
2486,6-Jan-22,138.8930,1.393340,0.0,0.0,11.764706,5.7036,1.3584,1.2741,841.63,...,0.91780,33.372,6.7824,0.0,3.6725,0.738716,1,0.000,2022,0.0
2487,7-Jan-22,139.3305,1.396258,0.0,0.0,11.695906,5.6747,1.3588,1.2666,838.88,...,0.92215,33.600,6.7811,0.0,0.0000,0.737953,1,44.515,2022,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643,22-Aug-22,141.0594,1.450958,0.0,0.0,12.787724,5.1703,1.3927,1.3035,945.35,...,0.95870,35.938,6.7687,0.0,3.6725,0.847170,1,40.431,2022,0.0
2644,23-Aug-22,141.2180,1.454122,0.0,0.0,12.836970,5.1024,1.3971,1.2972,945.47,...,0.96670,36.226,6.7560,0.0,3.6725,0.850051,1,40.369,2022,0.0
2645,24-Aug-22,140.9474,1.448226,0.0,0.0,12.836970,5.1044,1.3955,1.2983,926.94,...,0.96400,36.162,6.7701,0.0,3.6725,0.849185,1,40.401,2022,0.0
2646,25-Aug-22,140.6518,1.437815,0.0,0.0,12.738853,5.1167,1.3916,1.2937,915.60,...,0.96380,35.894,6.7554,0.0,3.6725,0.845594,1,0.000,2022,0.0


In [57]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%d-%b-%y')

ValueError: time data "2023-01-03" doesn't match format "%d-%b-%y", at position 2648. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [53]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming combined_df is already created and contains your data

# MySQL database connection details
username = 'root'
password = 'beansbestcat'
host = 'localhost'
port = '3307'  # Specify the port here
dbname = 'nt-t8-db'
table_name = 'historical_data'

# Create a connection string
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{dbname}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to the MySQL table
combined_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print("DataFrame written to MySQL database successfully.")

DataFrame written to MySQL database successfully.
